<a href="https://colab.research.google.com/github/tsanoop887-hash/AIF360/blob/main/word_level_text_generator_using_Rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM,Dropout,Dense
from tensorflow.keras.utils import to_categorical
import random



In [6]:
text = """Once upon a time in a faraway land there lived a kind and wise king.
He loved his people and ruled with fairness and compassion.
Every day, he walked among them and listened to their stories."""

In [5]:
text=text.lower()

tokenizer=Tokenizer()
tokenizer.fit_on_texts([text])

encoded=tokenizer.texts_to_sequences([text])[0]
vocab_size=len(tokenizer.word_index)+1

print(f"Vocabulary Size : {vocab_size}")

seq_length=5
x,y=[],[]
for i in range(seq_length,len(encoded)):
    x.append(encoded[i-seq_length:i])
    y.append(encoded[i])

X=np.array(x)
y= to_categorical(y,num_classes=vocab_size)

print(f"Training sequences: {X.shape}")

Vocabulary Size : 31
Training sequences: (31, 5)


In [4]:
text = """Once upon a time in a faraway land there lived a kind and wise king.
He loved his people and ruled with fairness and compassion.
Every day, he walked among them and listened to their stories."""

In [9]:
model=Sequential([
    Embedding(vocab_size,100,input_length=seq_length),
    LSTM(256,return_sequences=True),
    Dropout(0.3),
    LSTM(256),
    Dense(vocab_size,activation='softmax')
])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [10]:
history=model.fit(X,y,epochs=200,verbose=1)

Epoch 1/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.0000e+00 - loss: 3.4345
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step - accuracy: 0.1613 - loss: 3.4282
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step - accuracy: 0.1935 - loss: 3.4215
Epoch 4/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step - accuracy: 0.1613 - loss: 3.4145
Epoch 5/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 301ms/step - accuracy: 0.1613 - loss: 3.4054
Epoch 6/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step - accuracy: 0.1613 - loss: 3.3953
Epoch 7/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step - accuracy: 0.1613 - loss: 3.3824
Epoch 8/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 296ms/step - accuracy: 0.1613 - loss: 3.3655
Epoch 9/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step - accuracy: 0.1935 - loss: 3.3430
Epoch 10/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - accuracy: 0.1613 - loss: 3.3162
Epoch 11/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step - accuracy: 0.1935 - loss: 3.2812
Epoch 12/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - accuracy: 

In [11]:
def sample_with_temperature(preds,temperature=1.0):
  preds=np.asarray(preds).astype('float64')
  preds=np.log(preds+1e-8) / temperature
  exp_preds=np.exp(preds)
  preds=exp_preds/np.sum(exp_preds)
  return np.random.choice(len(preds),p=preds)

In [12]:
def generate_text(seed_text, num_words=20, temperature=1.0):
    result = seed_text
    for _ in range(num_words):
        token_list = tokenizer.texts_to_sequences([result.split()[-seq_length:]])[0]
        token_list = pad_sequences([token_list], maxlen=seq_length, padding='pre')
        preds = model.predict(token_list, verbose=0)[0]
        next_index = sample_with_temperature(preds, temperature)
        for word, index in tokenizer.word_index.items():
            if index == next_index:
                result += ' ' + word
                break
    return result


In [13]:
seed_text = "the king"
print("\n🌡️ Temperature = 0.5")
print(generate_text(seed_text, num_words=25, temperature=0.5))

print("\n🌡️ Temperature = 1.0")
print(generate_text(seed_text, num_words=25, temperature=1.0))

print("\n🌡️ Temperature = 1.5")
print(generate_text(seed_text, num_words=25, temperature=1.5))


🌡️ Temperature = 0.5
the king and and and and and his loved his his his with with compassion compassion compassion day day among among and and and their stories his

🌡️ Temperature = 1.0
the king and and and and ruled his loved his compassion fairness fairness with compassion he king he day them and and and and stories his loved

🌡️ Temperature = 1.5
the king and and and and and his loved his his his compassion with compassion his compassion walked loved among people them listened listened and and their
